<a href="https://colab.research.google.com/github/gdtan02/CV_Assignment_LLIE/blob/main/notebooks/ZeroDCE_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Zero-DCE Testing

## Environment Setup

Connect to Tesla T4 runtime before running any code block below.

In [8]:
!nvidia-smi

Sun Dec 29 08:11:08 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [9]:
!git clone https://github.com/gdtan02/CV_Assignment_LLIE.git

Cloning into 'CV_Assignment_LLIE'...
remote: Enumerating objects: 251, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 251 (delta 3), reused 10 (delta 3), pack-reused 238 (from 1)
Receiving objects: 100% (251/251), 30.96 MiB | 20.48 MiB/s, done.
Resolving deltas: 100% (142/142), done.


In [10]:
%cd CV_Assignment_LLIE/

/content/CV_Assignment_LLIE


In [11]:
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.9/106.9 kB 4.3 MB/s eta 0:00:00


In [12]:
!pip install torchvision

## Importing Dark Face Dataset and pretrained models

In [13]:
import os
import torch
import torch.nn as nn
import torchvision
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from random import shuffle
from glob import glob
from zero_dce import (
    download_test_dataset,
    Trainer, plot_result,
    NoisyImageDataset
)
from torch.utils.data import DataLoader
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import mean_squared_error as mse

mpl.rcParams['figure.max_open_warning'] = 0
%matplotlib inline

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [14]:
download_test_dataset()

Dataset file path =  /content/CV_Assignment_LLIE/data/DarkPair.zip


Downloading...
From (original): https://drive.google.com/uc?id=11KaOhxcOh68_NyZwacBoabEJ6FgPCsnQ
From (redirected): https://drive.google.com/uc?id=11KaOhxcOh68_NyZwacBoabEJ6FgPCsnQ&confirm=t&uuid=d790ea0f-0c54-4532-80fa-f4a776f1ba98
To: /content/CV_Assignment_LLIE/data/DarkPair.zip
100%|██████████| 580M/580M [00:09<00:00, 63.4MB/s]


Unzip the Dark Face dataset...
Done.


In [15]:
image_path = os.path.join(os.getcwd(), "data", "Low", "*.png")
image_files = glob(image_path)

if image_files is not None:
  print("Image files have been loaded.")
  print(f"Number of images: {len(image_files)}")
else:
  print("No image files found.")

Image files have been loaded.
Number of images: 789


In [16]:
trainer = Trainer()
trainer.build_model(pretrain_weights='models/checkpoints/model200_dark_faces.pth')
trainer.build_dae(pretrain_weights='models/checkpoints/dae_model_epoch_50.pth')

Model built successfully
DAE built successfully


/content/CV_Assignment_LLIE/zero_dce/trainer.py:158: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.model.load_state_dict(torch.load(pretrain_weights))
/content/CV_Assig

In [17]:
def evaluate_model(trainer, model, image_files):

  enhanced_images = []
  org_images = []

  psnr_values = []
  ssim_values = []
  mse_values = []

  for image in image_files:
    org_image, enhanced_image = trainer.evaluate(model, image)
    plot_result(org_image, enhanced_image, notebook=True)

    # Convert org_image to numpy array and remove the batch dimension
    org_image = org_image.squeeze(0).permute(1, 2, 0).cpu().numpy()

    enhanced_images.append(enhanced_image)
    org_images.append(org_image)

    psnr_value = psnr(org_image, enhanced_image, data_range=255)
    ssim_value = ssim(org_image, enhanced_image, data_range=255, win_size=3, channel_axis=-1)
    mse_value = mse(org_image, enhanced_image)

    psnr_values.append(psnr_value)
    ssim_values.append(ssim_value)
    mse_values.append(mse_value)

  return enhanced_images, org_images, psnr_values, ssim_values, mse_values


In [18]:
enhanced_images, org_images, _, _, _ = evaluate_model(trainer, trainer.model, image_files[:10])

Output hidden; open in https://colab.research.google.com to view.

In [19]:
print(type(enhanced_images[0]), enhanced_images[0].shape)
print(type(org_images[0]), org_images[0].shape)

<class 'numpy.ndarray'> (256, 256, 3)
<class 'numpy.ndarray'> (256, 256, 3)


## DAE Evaluation

In [20]:
noisy_dataset = NoisyImageDataset(img_files=image_files)

image_loader = DataLoader(
    noisy_dataset,
    shuffle=False,
    num_workers=4,
    batch_size=8,
)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [21]:
def evaluate_dae(trainer, image_loader, device):

  denoised_images = []
  clean_images = []
  noisy_images = []

  trainer.dae.to(device)

  trainer.dae.eval()

  criterion = nn.MSELoss(reduction='sum').to(device)

  with torch.no_grad():
    pred_loss = 0.0

    for noisy_image, clean_image in image_loader:
      noisy_image = noisy_image.to(device)
      clean_image = clean_image.to(device)

      denoised_image = trainer.dae(noisy_image)

      loss = criterion(denoised_image, clean_image)

      pred_loss += loss.item()

      # Process each image in the batch
      for i in range(denoised_image.size(0)):
          # Convert tensor to numpy for each image
          noisy_images.append(noisy_image[i].cpu().permute(1, 2, 0).numpy())
          denoised_images.append(denoised_image[i].cpu().permute(1, 2, 0).numpy())
          clean_images.append(clean_image[i].cpu().permute(1, 2, 0).numpy())

    pred_loss /= len(image_loader.dataset)

    print(f"Prediction loss: {pred_loss:.4f}")

  return noisy_images, denoised_images, clean_images, pred_loss


In [22]:
noisy_images, denoised_images, clean_images, pred_loss = evaluate_dae(trainer, image_loader, device)

Prediction loss: 85.5600


## Combined Results

In [23]:
# Convert the list of enhanced images into numpy array before converting into Tensor
enhanced_images = np.array(enhanced_images)
enhanced_images = torch.from_numpy(enhanced_images)
print(enhanced_images.shape)

torch.Size([10, 256, 256, 3])


In [24]:
new_enhanced_images = enhanced_images.permute(0, 3, 1, 2)
print(new_enhanced_images.shape)

torch.Size([10, 3, 256, 256])


In [25]:
def denoise(trainer, enhanced_images, device):

  # PSNR, SSIM, MSE Metrics (compare denoised image with original images, not enhanced images) -- TODO: Rain

  denoised_images = []

  trainer.dae.to(device)

  trainer.dae.eval()

  with torch.no_grad():
    for enhanced_image in enhanced_images:
      enhanced_image = enhanced_image.unsqueeze(0)
      enhanced_image = enhanced_image.to(device)

      denoised_image = trainer.dae(enhanced_image)

      denoised_images.append(denoised_image.cpu().squeeze(0).permute(1, 2, 0).numpy())

  return denoised_images

In [26]:
final_denoised_images = denoise(trainer, new_enhanced_images, device)

In [27]:
print(enhanced_images[0].shape)
print(final_denoised_images[0].shape)

torch.Size([256, 256, 3])
(256, 256, 3)


In [28]:
for i in range(len(final_denoised_images)):
  enhanced_image = np.array(enhanced_images[i])

  # may compare the metrics at here -- TODO: Rain

  plot_result(enhanced_image, final_denoised_images[i], notebook=True)

Output hidden; open in https://colab.research.google.com to view.

In [29]:
# for i in range(len(final_denoised_images)):
#   plot_result(enhanced_images[0], final_denoised_images[0], notebook=True)

##Evaluation Matrix

In [30]:
def compute_image_metrics(original_images, modified_images):
    """
    Compute PSNR, SSIM, and MSE metrics for a list of original and modified images.

    Args:
        original_images (list of torch.Tensor or np.ndarray): List of original images (H x W x C).
        modified_images (list of torch.Tensor or np.ndarray): List of modified images (H x W x C).

    Returns:
        metrics (dict): Dictionary containing averaged PSNR, SSIM, and MSE values.
        individual_scores (list of dict): List of individual metric scores for each image pair.
    """
    import numpy as np

    if len(original_images) != len(modified_images):
        raise ValueError("The number of original and modified images must match.")

    psnr_scores = []
    ssim_scores = []
    mse_scores = []

    for original, modified in zip(original_images, modified_images):
        # Convert PyTorch tensors to NumPy arrays if needed
        if isinstance(original, torch.Tensor):
            original = original.cpu().numpy()
        if isinstance(modified, torch.Tensor):
            modified = modified.cpu().numpy()

        # Ensure both inputs have the same dtype and range
        original = original.astype(np.float32)
        modified = modified.astype(np.float32)

        if original.shape != modified.shape:
            raise ValueError("Original and modified images must have the same dimensions.")

        # Compute metrics with explicit data_range
        psnr_val = psnr(original, modified, data_range=1.0)  # Assuming normalized [0, 1] images
        ssim_val = ssim(
            original,
            modified,
            win_size=min(7, original.shape[0], original.shape[1]),  # Ensure win_size is valid
            channel_axis=-1,
            data_range=1.0  # Assuming normalized [0, 1] images
        )
        mse_val = mse(original, modified)

        psnr_scores.append(psnr_val)
        ssim_scores.append(ssim_val)
        mse_scores.append(mse_val)

    # Aggregate metrics
    metrics = {
        "PSNR": np.mean(psnr_scores),
        "SSIM": np.mean(ssim_scores),
        "MSE": np.mean(mse_scores),
    }

    individual_scores = [
        {"PSNR": p, "SSIM": s, "MSE": m}
        for p, s, m in zip(psnr_scores, ssim_scores, mse_scores)
    ]

    return metrics, individual_scores


In [31]:
# Compute matrix between original images with enhanced images (ZERO_DEC)
metrics_ori_enhanced, individual_scores_ori_enhanced = compute_image_metrics(org_images, enhanced_images)

In [32]:
print("Averaged Metrics: Original images vs Enhanced images (ZERO_DEC)\n")
for metric, value in metrics_ori_enhanced.items():
    print(f"{metric}: {value:.4f}")

Averaged Metrics: Original images vs Enhanced images (ZERO_DEC)

PSNR: 6.1293
SSIM: 0.1149
MSE: 0.2533


In [33]:
print("\nIndividual Image Metrics: Original images vs Enhanced images (ZERO_DEC)\n")
for i, scores in enumerate(individual_scores_ori_enhanced):
    print(f"Image {i + 1}: {scores}")


Individual Image Metrics: Original images vs Enhanced images (ZERO_DEC)

Image 1: {'PSNR': 5.582621198575637, 'SSIM': 0.10161259, 'MSE': 0.27652721525798657}
Image 2: {'PSNR': 5.272944414483777, 'SSIM': 0.11968317, 'MSE': 0.2969651994844966}
Image 3: {'PSNR': 5.49134100850309, 'SSIM': 0.079415455, 'MSE': 0.28240078470327035}
Image 4: {'PSNR': 5.862995571618051, 'SSIM': 0.12273034, 'MSE': 0.2592390628679766}
Image 5: {'PSNR': 9.720946468336514, 'SSIM': 0.03270439, 'MSE': 0.10663637007171427}
Image 6: {'PSNR': 5.528296343298598, 'SSIM': 0.08977318, 'MSE': 0.280007952348015}
Image 7: {'PSNR': 6.206256888652431, 'SSIM': 0.14763613, 'MSE': 0.23953794043255763}
Image 8: {'PSNR': 7.005333062206141, 'SSIM': 0.06629225, 'MSE': 0.1992813670612629}
Image 9: {'PSNR': 4.822475565117871, 'SSIM': 0.31729054, 'MSE': 0.3294218816098405}
Image 10: {'PSNR': 5.799803503931452, 'SSIM': 0.07181231, 'MSE': 0.2630387000778403}


In [34]:
# Compute matrix between original images with final denoised images (ZERO_DEC + DAE)
metrics_ori_denoised, individual_scores_ori_denoised = compute_image_metrics(org_images, final_denoised_images)

In [35]:
print("Averaged Metrics: Original images vs Final denoised images (ZERO_DEC + DAE)\n")
for metric, value in metrics_ori_denoised.items():
    print(f"{metric}: {value:.4f}")

Averaged Metrics: Original images vs Final denoised images (ZERO_DEC + DAE)

PSNR: 6.2181
SSIM: 0.1208
MSE: 0.2488


In [36]:
print("\nIndividual Image Metrics: Original images vs Final denoised images (ZERO_DEC + DAE)\n")
for i, scores in enumerate(individual_scores_ori_denoised):
    print(f"Image {i + 1}: {scores}")


Individual Image Metrics: Original images vs Final denoised images (ZERO_DEC + DAE)

Image 1: {'PSNR': 5.666172856728924, 'SSIM': 0.107431166, 'MSE': 0.2712580992944626}
Image 2: {'PSNR': 5.381494005167076, 'SSIM': 0.12926452, 'MSE': 0.28963470514768613}
Image 3: {'PSNR': 5.537255573824687, 'SSIM': 0.101597816, 'MSE': 0.2794309084123749}
Image 4: {'PSNR': 5.938574539333788, 'SSIM': 0.12402334, 'MSE': 0.2547666321740678}
Image 5: {'PSNR': 9.966225538709011, 'SSIM': 0.05392309, 'MSE': 0.10078071754052272}
Image 6: {'PSNR': 5.562088288736513, 'SSIM': 0.097061396, 'MSE': 0.2778376974355948}
Image 7: {'PSNR': 6.308027464577207, 'SSIM': 0.12896784, 'MSE': 0.2339899763593102}
Image 8: {'PSNR': 7.104721457648121, 'SSIM': 0.07459444, 'MSE': 0.1947725966743509}
Image 9: {'PSNR': 4.874556550863896, 'SSIM': 0.29280615, 'MSE': 0.3254950174260358}
Image 10: {'PSNR': 5.8418469121925245, 'SSIM': 0.0987969, 'MSE': 0.26050454738810913}
